In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_result = "/content/drive/MyDrive/ICPS/"
path_model ="/content/drive/MyDrive/ICPS/"

In [ ]:
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score
import os

num_folds = 5
num_epochs = 50
lr = 1e-3
result_CNN_BiLSTM = "results_cross_cnn_bilstm.txt"
if os.path.exists(result_CNN_BiLSTM):
    os.remove(result_CNN_BiLSTM)

kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
fold_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train), 1):
    print(f"\n🔁 Fold {fold}/{num_folds}")

    X_tr, X_val = X_train[train_idx], X_train[val_idx]
    y_tr, y_val = y_train[train_idx], y_train[val_idx]

    model = Classifier_CNN_BiLSTM_Optimized(MAX_SEQUENCE_LENGTH, feature_dim, lr=lr)

    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
    checkpoint = ModelCheckpoint(f"best_model_fold_{fold}.h5", monitor='val_loss', save_best_only=True, verbose=1)

    model.fit(
        [X_tr, X_tr], y_tr,
        validation_data=([X_val, X_val], y_val),
        batch_size=batch_size,
        epochs=num_epochs,
        callbacks=[early_stop, checkpoint],
        verbose=1,
        shuffle=True
    )

    y_pred_prob = model.predict([X_val, X_val])
    y_pred = (y_pred_prob > 0.5).astype(int).flatten()
    cm = confusion_matrix(y_val, y_pred)
    mcc = matthews_corrcoef(y_val, y_pred)
    acc = accuracy_score(y_val, y_pred)
    sn = cm[1][1] / (cm[1][1] + cm[1][0]) if (cm[1][1] + cm[1][0]) else 0
    sp = cm[0][0] / (cm[0][0] + cm[0][1]) if (cm[0][0] + cm[0][1]) else 0

    fold_scores.append({'accuracy': acc, 'mcc': mcc, 'sn': sn, 'sp': sp})

    print(f"✅ Fold {fold} - ACC: {acc:.4f}, MCC: {mcc:.4f}, SN: {sn:.4f}, SP: {sp:.4f}")

# Tổng hợp kết quả
avg_acc = np.mean([s['accuracy'] for s in fold_scores])
avg_mcc = np.mean([s['mcc'] for s in fold_scores])
avg_sn = np.mean([s['sn'] for s in fold_scores])
avg_sp = np.mean([s['sp'] for s in fold_scores])

print("\n📊 === Final CV Results ===")
print(f"Avg Accuracy:     {avg_acc:.4f}")
print(f"Avg MCC:          {avg_mcc:.4f}")
print(f"Avg Sensitivity:  {avg_sn:.4f}")
print(f"Avg Specificity:  {avg_sp:.4f}")